In [11]:
def parse_single_row_to_records(row):
    cate = row['category']
    question = row['question']
    answer = row['answer']
    sheet = row['sheet']

    questions = [q.strip() for q in question.split('\n') if len(q.strip())]
    cate_name = questions[0][:20]
    records = [{'category': cate, 'cate': f'{cate}-{cate_name}', 'question': q, 'answer': answer, 'sheet': sheet} for q in questions]
    return records

In [12]:
# out = parse_single_row_to_records(df.iloc[1].to_dict())
# print(len(out))
# pd.DataFrame(out).tail()

In [37]:
records = []
for idx, row in ori_df.iterrows():
    _rec = parse_single_row_to_records(row)
    records += _rec

df = pd.DataFrame(records)
df.shape

(466, 5)

In [38]:
df[:5]

,category,cate,question,answer,sheet
0,fallback,fallback-fallback,fallback,[[[Flexible2]]] $otext,flexible1
1,menudriven-01_checking,menudriven-01_checking-ตรวจสอบเครดิตบูโร,ตรวจสอบเครดิตบูโร,"[[TEMPLATE]] {\n ""type"": ""carousel"",\n ""cont...",flexible1
2,menudriven-01_checking,menudriven-01_checking-ตรวจสอบเครดิตบูโร,ตรวจเครดิตบูโร,"[[TEMPLATE]] {\n ""type"": ""carousel"",\n ""cont...",flexible1
3,menudriven-01_checking,menudriven-01_checking-ตรวจสอบเครดิตบูโร,ตรวจสอบ,"[[TEMPLATE]] {\n ""type"": ""carousel"",\n ""cont...",flexible1
4,menudriven-01_checking,menudriven-01_checking-ตรวจสอบเครดิตบูโร,เครดิตบูโรอย่างไร,"[[TEMPLATE]] {\n ""type"": ""carousel"",\n ""cont...",flexible1


In [39]:
df.cate.value_counts()

cate
faq_055-ตรวจเครดิตบูโรได้ที่                      22
menudriven-01_checking-ตรวจสอบเครดิตบูโร          18
faq_021-สิทธิตรวจสอบสถานะ แก                      16
faq_019-ตววจแล้วไม่ได้                            15
menudriven-02_invalid_credit-มีประวัติค้างชำระ    13
                                                  ..
faq_115-เมื่อไหร่จะมีแอป                           1
faq_072-แบบฟอร์มคำขอตรวจสอบค                       1
faq_117-แอปทางรัฐ คือ                              1
faq_095-ทำไมไม่ไปงาน Money E                       1
faq_082-ขอรายละเอียดเอกสารใน                       1
Name: count, Length: 126, dtype: int64

# 01 LOAD DATA

In [54]:
from llama_index.core import SimpleDirectoryReader

In [55]:
reader = SimpleDirectoryReader(input_dir="./data")
documents = reader.load_data()

In [56]:
documents[0]

Document(id_='9611675c-6572-4d2f-b858-c69ce0b0a9e1', embedding=None, metadata={'file_path': '/home/prejudice/workspace/PROJECT/stand_alone/workshop/rag-chatbot/data/01-snake-different.txt', 'file_name': '01-snake-different.txt', 'file_type': 'text/plain', 'file_size': 4638, 'creation_date': '2025-01-30', 'last_modified_date': '2025-01-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='งูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก ดูแค่ส่วนหัวก็รู้เลย\n10 ส.ค. 67 (18:01 น.) พิมพ์\nงูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก ดูแค่ส่วนหัวก็รู้เลย\nแชร์เรื่องนี้\nคัดลอกลิงก์\nกดฟัง\n-ก\nก+\nงูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก 3 ข้อ ซึ่งจริงๆ ดูแค่ส่วนหัวก็เห็นความแตกต่างแล้ว\n\nงูเหลือม และ งูหลา

# 02 INDEXING

#### CONVERT DOCS TO NODES

In [6]:
from llama_index.core.node_parser import TokenTextSplitter

In [65]:
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    separator=" ",
)

In [66]:
nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
len(documents), len(nodes)

(3, 30)

#### SET LLM ENV

In [68]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [69]:
embedding_model_name = 'BAAI/bge-m3'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024)

In [70]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


#### INDEXING

In [71]:
import faiss
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Document,
)        
from llama_index.vector_stores.faiss import FaissVectorStore

In [72]:
dim = 1024
faiss_index = faiss.IndexFlatIP(dim)

In [73]:
%%time
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes=nodes,storage_context=storage_context)

CPU times: user 2min 7s, sys: 1.62 s, total: 2min 8s
Wall time: 22.6 s


##### SAVE INDEX

In [74]:
persist_dir = 'index'
# persist_dir = 'index/nstda-faq'

index.storage_context.persist(persist_dir=persist_dir)

In [75]:
!tree index/

index/
├── default__vector_store.json
├── docstore.json
├── graph_store.json
├── image__vector_store.json
└── index_store.json

0 directories, 5 files


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 03 SEARCH

In [76]:
from llama_index.core import (

    load_index_from_storage,
    StorageContext,
)

In [77]:
persist_dir="index"

vector_store = FaissVectorStore.from_persist_dir(persist_dir)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context)

In [78]:
retriever = index.as_retriever(similarity_top_k=5)

In [83]:
text = 'เราสามารถแยกงูเหลือมกับงูหลามออกจากกันได้ไง'

response = retriever.retrieve(text)
print(f'Query: {text}', '\n')
for i in range(3):
    print('Docid:', response[i].metadata.get('docid', ''))
    print('Response:', response[i].text)
    print('Score:', response[i].score)
    print('Metadata', response[i].metadata)
    print('-'*50,'\n')

Query: เราสามารถแยกงูเหลือมกับงูหลามออกจากกันได้ไง 

Docid: 
Response: งูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก ดูแค่ส่วนหัวก็รู้เลย
10 ส.ค. 67 (18:01 น.) พิมพ์
งูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก ดูแค่ส่วนหัวก็รู้เลย
แชร์เรื่องนี้
คัดลอกลิงก์
กดฟัง
-ก
ก+
งูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก 3 ข้อ ซึ่งจริงๆ ดูแค่ส่วนหัวก็เห็นความแตกต่างแล้ว

งูเหลือม และ งูหลาม เป็นงูไม่มีพิษขนาดใหญ่ที่พบได้บ่อยในประเทศไทย โดยทั้ง 2 ชนิด ปัจจุบันเป็นสัตว์ป่าคุ้มครอง ตามพระราชบัญญัติสงวนและคุ้มครองสัตว์ป่า
Score: 0.7475630640983582
Metadata {'file_path': '/home/prejudice/workspace/PROJECT/stand_alone/workshop/rag-chatbot/data/01-snake-different.txt', 'file_name': '01-snake-different.txt', 'file_type': 'text/plain', 'file_size': 4638, 'creation_date': '2025-01-30', 'last_modified_date': '2025-01-30'}
-------------------------------------------------- 

Docid: 
Response: โดยทั้ง 2 ชนิด ปัจจุบันเป็นสัตว์ป่าคุ้มครอง ตามพระราชบัญญัติสงวน

In [80]:
query_engine = index.as_query_engine(similarity_top_k=3)

In [81]:
res = query_engine.query(text)

In [82]:
print(res.response)

Context information is below.
---------------------
file_path: /home/prejudice/workspace/PROJECT/stand_alone/workshop/rag-chatbot/data/01-snake-different.txt

งูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก ดูแค่ส่วนหัวก็รู้เลย
10 ส.ค. 67 (18:01 น.) พิมพ์
งูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก ดูแค่ส่วนหัวก็รู้เลย
แชร์เรื่องนี้
คัดลอกลิงก์
กดฟัง
-ก
ก+
งูเหลือม กับ งูหลาม ต่างกันยังไง? เพิ่งรู้มีวิธีสังเกตง่ายมาก 3 ข้อ ซึ่งจริงๆ ดูแค่ส่วนหัวก็เห็นความแตกต่างแล้ว

งูเหลือม และ งูหลาม เป็นงูไม่มีพิษขนาดใหญ่ที่พบได้บ่อยในประเทศไทย โดยทั้ง 2 ชนิด ปัจจุบันเป็นสัตว์ป่าคุ้มครอง ตามพระราชบัญญัติสงวนและคุ้มครองสัตว์ป่า

file_path: /home/prejudice/workspace/PROJECT/stand_alone/workshop/rag-chatbot/data/01-snake-different.txt

โดยทั้ง 2 ชนิด ปัจจุบันเป็นสัตว์ป่าคุ้มครอง ตามพระราชบัญญัติสงวนและคุ้มครองสัตว์ป่า พุทธศักราช 2535

ความแตกต่างของงูเหลือมและงูหลาม คือ งูหลามมีลำตัวที่ใหญ่อ้วนป้อมกว่างูเหลือม แต่มีความยาวน้อยกว่างูเหลือม ความยาวของงูหลามอยู่ที่ประมาณ 1